In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **1. Install necessary libraries (PEFT, datasets) :**

In [1]:
!pip install --q peft==0.4.0

In [111]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
import peft
from peft import LoraConfig, get_peft_model, PeftModel
import torch
import pandas as pd
import numpy as np
import transformers
from transformers import TrainingArguments, Trainer
import os
from IPython.display import display

# **Load a pre-trained language model (bigscience/bloomz-560m) and its tokenizer :**

In [3]:
model_name = "bigscience/bloomz-560m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# **Load the dataset and preprocess it for the model :**

In [150]:
data = load_dataset("Abirate/english_quotes", split="train[:10%]")

In [79]:
type(data)

datasets.arrow_dataset.Dataset

In [80]:
print(data)

Dataset({
    features: ['quote', 'author', 'tags'],
    num_rows: 251
})


In [81]:
data[0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator']}

In [151]:
data = data.map(lambda x: tokenizer(x["quote"], padding=True, truncation=True), batched=True)

Map:   0%|          | 0/251 [00:00<?, ? examples/s]

In [152]:
data = data.remove_columns(["quote", "author", "tags"])

In [153]:
data[0]

{'input_ids': [3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  

# **Configure LoRA using LoraConfig :**

In [50]:
for name, module in model.named_modules():
  print("name :", name)

name : 
name : transformer
name : transformer.word_embeddings
name : transformer.word_embeddings_layernorm
name : transformer.h
name : transformer.h.0
name : transformer.h.0.input_layernorm
name : transformer.h.0.self_attention
name : transformer.h.0.self_attention.query_key_value
name : transformer.h.0.self_attention.dense
name : transformer.h.0.self_attention.attention_dropout
name : transformer.h.0.post_attention_layernorm
name : transformer.h.0.mlp
name : transformer.h.0.mlp.dense_h_to_4h
name : transformer.h.0.mlp.gelu_impl
name : transformer.h.0.mlp.dense_4h_to_h
name : transformer.h.1
name : transformer.h.1.input_layernorm
name : transformer.h.1.self_attention
name : transformer.h.1.self_attention.query_key_value
name : transformer.h.1.self_attention.dense
name : transformer.h.1.self_attention.attention_dropout
name : transformer.h.1.post_attention_layernorm
name : transformer.h.1.mlp
name : transformer.h.1.mlp.dense_h_to_4h
name : transformer.h.1.mlp.gelu_impl
name : transforme

In [154]:
lora_config = LoraConfig(
    r=1,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# **5. Apply LoRA to the pre-trained model using get_peft_model :**

In [155]:
peft_model = get_peft_model(model, lora_config)

# **6. Set up training arguments using TrainingArguments :**

In [156]:
output_directory = os.path.join("/content/drive/MyDrive/GENAI/Week7/Day2", "peft_lab_outputs")
training_args = TrainingArguments(
    report_to="none",
    output_dir=output_directory,
    auto_find_batch_size=True,
    learning_rate= 3e-5,
    num_train_epochs=5,
    use_cpu=False
)

# **7. Initialize and train the model using Trainer :**

In [157]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [158]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=315, training_loss=3.2785733661954364, metrics={'train_runtime': 559.1838, 'train_samples_per_second': 2.244, 'train_steps_per_second': 0.563, 'total_flos': 1243329727610880.0, 'train_loss': 3.2785733661954364, 'epoch': 5.0})

# **8. Save the fine-tuned LoRA model :**

In [105]:
peft_model_path = os.path.join(output_directory, "peft_model")
peft_model.save_pretrained(peft_model_path)

# **9. Load the saved LoRA model for inference using PeftModel.from_pretrained :**

In [112]:
peft_model = PeftModel.from_pretrained(model, peft_model_path)

# **10. Generate text using the fine-tuned model and the tokenizer :**

In [160]:
inputs = tokenizer("Two things are infinite: ", return_tensors="pt")
inputs = {k: v.to(peft_model.device) for k, v in inputs.items()}

outputs = peft_model.generate(
    **inputs,
    max_length=50,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    eos_token_id=tokenizer.eos_token_id
    )

print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['Two things are infinite:  time and space. But there is one thing that is not infinite: time itself.']
